<a href="https://colab.research.google.com/github/sonu786786/Responsible_AI/blob/main/Lab_03/quest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kagglehub


In [3]:
import os
os.environ["KAGGLE_API_TOKEN"] = "KGAT_155b95a19a48a3e19bfe45f85966a4a0"


In [4]:
import kagglehub

path = kagglehub.dataset_download("ryanbadai/clothes-dataset")
print("Dataset path:", path)


100%|██████████| 1.37G/1.37G [00:49<00:00, 29.9MB/s]

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/ryanbadai/clothes-dataset/versions/1


In [5]:
import os
os.listdir(path)


['Clothes_Dataset']

In [6]:
dataset_path = os.path.join(path, "Clothes_Dataset")
os.listdir(dataset_path)


['Blazer',
 'Jaket_Denim',
 'Rok',
 'Celana_Panjang',
 'Kaos',
 'Jaket',
 'Mantel',
 'Celana_Pendek',
 'Polo',
 'Gaun',
 'Sweter',
 'Jeans',
 'Kemeja',
 'Hoodie',
 'Jaket_Olahraga']

In [7]:
for cls in os.listdir(dataset_path):
    cls_path = os.path.join(dataset_path, cls)
    print(cls, ":", len(os.listdir(cls_path)))


Blazer : 500
Jaket_Denim : 500
Rok : 500
Celana_Panjang : 500
Kaos : 500
Jaket : 500
Mantel : 500
Celana_Pendek : 500
Polo : 500
Gaun : 500
Sweter : 500
Jeans : 500
Kemeja : 500
Hoodie : 500
Jaket_Olahraga : 500


In [8]:
import cv2
import numpy as np

IMG_SIZE = 64
X, y = [], []

class_names = sorted(os.listdir(dataset_path))

for label, class_name in enumerate(class_names):
    class_path = os.path.join(dataset_path, class_name)
    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)

print("Images shape:", X.shape)
print("Labels shape:", y.shape)


Images shape: (7500, 64, 64)
Labels shape: (7500,)


In [9]:
X = X.reshape(X.shape[0], -1)
print("Flattened shape:", X.shape)


Flattened shape: (7500, 4096)


Each 2-D image is flattened into a 1-D feature vector so that logistic regression can treat each pixel as an independent feature.

In [10]:
X = X / 255.0


Normalization is necessary because logistic regression is sensitive to feature scales and normalization improves convergence.

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

model = OneVsRestClassifier(
    LogisticRegression(max_iter=1000)
)

model.fit(X_train, y_train)


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000))

In [14]:
y_pred = model.predict(X_test)


In [15]:
from sklearn.metrics import f1_score, classification_report

macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1 Score:", macro_f1)

print(classification_report(y_test, y_pred))


Macro F1 Score: 0.2610008257335125
              precision    recall  f1-score   support

           0       0.16      0.15      0.16       100
           1       0.28      0.24      0.26       100
           2       0.31      0.35      0.33       100
           3       0.21      0.18      0.19       100
           4       0.24      0.23      0.24       100
           5       0.30      0.31      0.30       100
           6       0.25      0.19      0.22       100
           7       0.23      0.25      0.24       100
           8       0.48      0.43      0.45       100
           9       0.29      0.35      0.32       100
          10       0.34      0.31      0.32       100
          11       0.26      0.25      0.25       100
          12       0.18      0.23      0.20       100
          13       0.19      0.18      0.18       100
          14       0.23      0.27      0.25       100

    accuracy                           0.26      1500
   macro avg       0.26      0.26      0.26  

How did you convert 2-D images into 1-D arrays?

The 2-D image matrices were flattened into 1-D vectors by concatenating pixel values row-wise, allowing each pixel to act as a feature for logistic regression.

Is normalization necessary?

Yes. Normalization is essential because logistic regression is sensitive to feature magnitudes. Scaling pixel values improves convergence and stability.

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


In [17]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)


In [18]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [21]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes, activation_fn):
        super(MLP, self).__init__()

        layers = []
        prev_size = input_size

        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(activation_fn)
            prev_size = hidden_size

        layers.append(nn.Linear(prev_size, num_classes))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [22]:
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()


In [23]:
from sklearn.metrics import f1_score, accuracy_score

def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            predictions = torch.argmax(outputs, dim=1)
            y_true.extend(y_batch.numpy())
            y_pred.extend(predictions.numpy())

    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, average='macro')

    return acc, f1


In [24]:
input_size = X_train.shape[1]
num_classes = len(set(y_train))

hidden_layer_configs = {
    "1_hidden": [256],
    "2_hidden": [256, 128],
    "3_hidden": [256, 128, 64]
}

activation_functions = {
    "ReLU": nn.ReLU(),
    "Sigmoid": nn.Sigmoid(),
    "Tanh": nn.Tanh()
}


In [25]:
results = []

for hl_name, hidden_layers in hidden_layer_configs.items():
    for act_name, act_fn in activation_functions.items():

        model = MLP(
            input_size=input_size,
            hidden_layers=hidden_layers,
            num_classes=num_classes,
            activation_fn=act_fn
        )

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        train_model(model, train_loader, criterion, optimizer, epochs=10)
        acc, f1 = evaluate_model(model, test_loader)

        results.append((hl_name, act_name, acc, f1))
        print(f"{hl_name} | {act_name} → Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")


1_hidden | ReLU → Accuracy: 0.3267, Macro F1: 0.3125
1_hidden | Sigmoid → Accuracy: 0.2793, Macro F1: 0.2727
1_hidden | Tanh → Accuracy: 0.1987, Macro F1: 0.1796
2_hidden | ReLU → Accuracy: 0.3233, Macro F1: 0.3219
2_hidden | Sigmoid → Accuracy: 0.2327, Macro F1: 0.2073
2_hidden | Tanh → Accuracy: 0.2053, Macro F1: 0.1763
3_hidden | ReLU → Accuracy: 0.3393, Macro F1: 0.3259
3_hidden | Sigmoid → Accuracy: 0.1687, Macro F1: 0.1264
3_hidden | Tanh → Accuracy: 0.1933, Macro F1: 0.1523


In [26]:
import pandas as pd

df = pd.DataFrame(results, columns=["Hidden Layers", "Activation", "Accuracy", "Macro F1"])
df


,Hidden Layers,Activation,Accuracy,Macro F1
0,1_hidden,ReLU,0.326667,0.312513
1,1_hidden,Sigmoid,0.279333,0.272674
2,1_hidden,Tanh,0.198667,0.179637
3,2_hidden,ReLU,0.323333,0.321864
4,2_hidden,Sigmoid,0.232667,0.207331
5,2_hidden,Tanh,0.205333,0.176269
6,3_hidden,ReLU,0.339333,0.325913
7,3_hidden,Sigmoid,0.168667,0.126354
8,3_hidden,Tanh,0.193333,0.152295


A Multi-Layer Perceptron was implemented using PyTorch with varying depths and activation functions. Experiments show that increasing the number of hidden layers improves performance, and ReLU activation achieves the best macro F1 score among the tested functions.